In [12]:

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys



In [13]:
#path = get_file('all_lyrics_no-umlaut_edited2.txt', origin='http://research.komakino.ch/all_lyrics_no-umlaut_edited2.txt ')

filename = 'all_lyrics_no-umlaut_edited2.txt'

text = open(filename).read().lower()

text = text.replace('\x99', '')
text = text.replace('\x80', '')
text = text.replace('\xe2', '')


text = text.replace('"', '')
text = text.replace('(', '')
text = text.replace(')', '')
text = text.replace('_', '')
text = text.replace(';', ',')
text = text.replace('&amp;', '&')



#all_lyrics_no-umlaut_edited2.txt
#path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
#text = open(path).read().lower()
print('corpus length:', len(text))
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
#step = 1
sentences = []
next_chars = []

corpus length: 149346
total chars: 47


In [14]:

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

nb sequences: 49769
Vectorization...


In [15]:
for (i, sentence) in enumerate(sentences):
    for (t, char) in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1

In [6]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)



Build model...


In [7]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)



In [17]:
filepath="aws_128_00_length40_step3_batch128_weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min', period=1)
callbacks_list = [checkpoint]

# Word_weights-improvement-00-0.5946.hdf5


# train the model, output generated text after each iteration
for iteration in range(1, 30):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    history = model.fit(X, y,
              batch_size=128,
              epochs=1, 
              callbacks=callbacks_list)

    

    start_index = random.randint(0, len(text) - maxlen - 1)
    if (iteration%10==0):
        for diversity in [0.1, 0.2, 0.5, 1.0]:
            print()
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index: start_index + maxlen]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            sys.stdout.write(generated)

            for i in range(200):
                x = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x[0, t, char_indices[char]] = 1.

                preds = model.predict(x, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]

                generated += next_char
                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()




--------------------------------------------------
Iteration 1
Epoch 1/1
49769/49769 [==============================] - 23s - loss: 0.9677    

--------------------------------------------------
Iteration 2
Epoch 1/1
49769/49769 [==============================] - 23s - loss: 0.9622    

--------------------------------------------------
Iteration 3
Epoch 1/1
49769/49769 [==============================] - 23s - loss: 0.9672    

--------------------------------------------------
Iteration 4
Epoch 1/1
49769/49769 [==============================] - 23s - loss: 0.9695    

--------------------------------------------------
Iteration 5
Epoch 1/1
49769/49769 [==============================] - 23s - loss: 0.9648    

--------------------------------------------------
Iteration 6
Epoch 1/1
49769/49769 [==============================] - 23s - loss: 0.9678    

--------------------------------------------------
Iteration 7
Epoch 1/1
49769/49769 [==============================] - 23s - loss: 0.9

/home/ubuntu/.local/lib/python2.7/site-packages/ipykernel/__main__.py:4: RuntimeWarning: divide by zero encountered in log


 sub the tay and the sania ci

----- diversity: 0.5
----- Generating with seed: "nyone
fight for money, fight for fun

"
nyone
fight for money, fight for fun

i have to be way you want to be on from the one but the night and they shock they to streating out of the douby
i'm a bout the sould briendybes all i look pretty word the for the night and you have t

----- diversity: 1.0
----- Generating with seed: "nyone
fight for money, fight for fun

"
nyone
fight for money, fight for fun

when you're in my to you
be til she have to no coaster, pirty
there's night
i had the gary, live, i got my head toke

time astay boy shosure but she was taphina?
treet there never not crazy, oothe

--------------------------------------------------
Iteration 11
Epoch 1/1
49769/49769 [==============================] - 33s - loss: 0.9683    

--------------------------------------------------
Iteration 12
Epoch 1/1
49769/49769 [==============================] - 33s - loss: 0.9633    

------------------------

KeyboardInterrupt: 